remake the network after subsetting all of the nodes in the data that we downloaded (above 10 commits or whatever)

In [15]:
%matplotlib inline

# load modules 
import warnings
from datetime import datetime
from text_unidecode import unidecode
from collections import deque
warnings.filterwarnings('ignore')

import os
import multiprocessing
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
import networkx as nx
from node2vec import Node2Vec

sns.set_style('whitegrid')

# connect to the database, download data 
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

edgelist_data = '''SELECT login, slug, commits as weight FROM gh.commits_per_user_dd_lchn LIMIT 1000'''

# convert to a dataframe, show how many missing we have (none)
edgelist_data = pd.read_sql_query(edgelist_data, con=connection)
edgelist_data.head()

,login,slug,weight
0,EugeneYilia,EugeneYilia/EugeneSignIn,511285
1,jthingelstad,jthingelstad/micro.thingelstad.com,352257
2,adamprocter,adamprocter/microblog,279801
3,curtclifton,curtclifton/curtclifton.github.io,253528
4,craigwmcclellan,craigwmcclellan/craigwmcclellan.github.io,181513


In [16]:
# convert the edgelist to a graph 
graph = nx.from_pandas_edgelist(edgelist_data, source='login', target='slug', edge_attr="weight")

In [17]:
cores_available = multiprocessing.cpu_count() - 1

# train the graph with node2vec 
print("Started at:", datetime.now())
node2vec = Node2Vec(graph, dimensions=20, walk_length=16, num_walks=100, workers=cores_available)
print("Finished at:", datetime.now())

Started at: 2021-07-11 17:59:24.379197


Computing transition probabilities:   0%|          | 0/1297 [00:00<?, ?it/s]

Finished at: 2021-07-11 18:01:23.268399


In [ ]:
# Running large graphs 

# https://github.com/eliorc/node2vec
# https://github.com/eliorc/node2vec/blob/master/example.py
    
node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4, temp_folder="/mnt/tmp_data")

In [18]:
model = node2vec.fit(window=10, min_count=1)

In [25]:
for node, _ in model.most_similar('curtclifton'):
    # Show only players
    if len(node) > 3:
        print(node)

curtclifton/curtclifton.github.io
mazeit/node-forum
AnghelLeonard
AnghelLeonard/Hibernate-SpringBoot
mazeit
bozhink
bozhink/ProcessingTools
fatalxiao
zhangluwei
alcedo-ui/alcedo-ui


In [21]:
edgelist_data

,login,slug,weight
0,EugeneYilia,EugeneYilia/EugeneSignIn,511285
1,jthingelstad,jthingelstad/micro.thingelstad.com,352257
2,adamprocter,adamprocter/microblog,279801
3,curtclifton,curtclifton/curtclifton.github.io,253528
4,craigwmcclellan,craigwmcclellan/craigwmcclellan.github.io,181513
...,...,...,...
995,mikepsinn,QuantiModo/quantimodo-android-chrome-ios-web-app,2020
996,nallar,MinimallyCorrect/TickThreading,2020
997,baldhumanity,baldhumanity/AcfunHated,2019
998,Lauriy,Ajapaik/ajapaik-web,2018


Future directions____ 

https://github.com/shenweichen/GraphEmbedding <<< 

node2vec 
struc2vec

[Node2Vec Tutorial](https://github.com/eliorc/Medium/blob/master/Nod2Vec-FIFA17-Example.ipynb)
